## Union de caracteristicas DTE y contribuyentes

En este notebook se hace la union final de los documentos seleccionados y las características de los contribuyentes involucrados en dicha transacción.

In [1]:
##Se importan packages necesarios
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import pyspark
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
from pyspark.sql.types import StringType,TimestampType
import matplotlib.pyplot as plt
from pyspark.sql import functions as F

In [2]:
#inicio de sesion en spark
ss_name = 'Lectura de datos Dashboard'
wg_conn = "spark.kerberos.access.hadoopFileSystems"
db_conn = "abfs://data@datalakesii.dfs.core.windows.net/"

spark = SparkSession.builder \
      .appName(f"Ejecucion algoritmo {ss_name}")  \
      .config(wg_conn, db_conn) \
      .config("spark.executor.memory", "6g") \
      .config("spark.driver.memory", "12g")\
      .config("spark.executor.cores", "4") \
      .config("spark.executor.instances", "5") \
      .config("spark.driver.maxResultSize", "12g") \
      .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

spark.conf.set("spark.sql.parquet.enableVectorizedReader","false")
spark.conf.set("spark.sql.parquet.int96RebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.parquet.int96RebaseModeInWrite", "CORRECTED")
spark.conf.set("spark.sql.parquet.datetimeRebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.parquet.datetimeRebaseModeInWrite", "CORRECTED")
spark.conf.set("spark.sql.debug.maxToStringFields", "2000")

Setting spark.hadoop.yarn.resourcemanager.principal to hvega.externo


## Lectura de datos

In [3]:
ruta_dtes = "abfs://data@datalakesii.dfs.core.windows.net/DatosOrigen/lr-629/APA/Analisis_factura/dtes_muestra_estratificada"

# Leer el DataFrame desde la ruta especificada
dte = spark.read.format("parquet").load(ruta_dtes)

In [4]:
# Ruta donde está el DataFrame de contribuyentes en formato Parquet
ruta_contribuyentes = "abfs://data@datalakesii.dfs.core.windows.net/DatosOrigen/lr-629/APA/Analisis_factura/data_contribuyentes"

# Leer el DataFrame desde la ruta especificada
contribuyentes = spark.read.format("parquet").load(ruta_contribuyentes)

In [5]:
contribuyentes.columns

['CONT_RUT',
 'CONT_DV',
 'INICIO_SEGMENTO',
 'ES_EMPRESA',
 'ES_PERSONA',
 'Alerta_1019',
 'Alerta_2250',
 'Alerta_400X',
 'Alerta_4110',
 'Alerta_4111',
 'Alerta_4112',
 'Alerta_4113',
 'Alerta_52',
 'Alerta_5201',
 'Alerta_5203',
 'Alerta_53',
 'Alerta_5301',
 'UNOP_UNIDAD_GRAN_CONT',
 'UNOP_COD_REGIONAL',
 'NEGO_IND_EXPORTADOR_VO',
 'NEGO_IND_PRIMERA_EXP_VO',
 'NEGO_IND_VERIFICACION_VO',
 'NEGO_NRO_FACTURAS_6MESES_VO',
 'COCO_IMP_VENTAS_IVA',
 'COCO_IMP_VENTAS_TRANSPORTE',
 'COCO_MCA_1_CATEGORIA',
 'COCO_MCA_2_CATEGORIA',
 'COCO_MCA_AFECTO_IMPTO_ADIC',
 'COCO_MCA_AFECTO_IMPTO_UNICO',
 'COCO_MCA_DOBLE_DECL_F22',
 'COCO_MCA_DONACIONES_CULTURALES',
 'COCO_MCA_DONACIONES_DEPORTIVAS',
 'COCO_MCA_DONACIONES_EDUCACIONALES',
 'COCO_MCA_DONACIONES_POLITICAS',
 'COCO_MCA_DONACIONES_UNIVERSIDAD',
 'COCO_MCA_ES_EMPRESA',
 'COCO_MCA_ES_GRAN_CONT',
 'COCO_MCA_ES_MINERA',
 'COCO_MCA_GLOBAL_COMPLE',
 'COCO_MCA_IMP_PPM_FONDO_MUTUO',
 'COCO_MCA_IMP_SOC_PROC',
 'COCO_MCA_SIN_CLAS_IMP',
 'COCO_MCA_TIP

In [6]:
dte.columns

['dhdr_rut_recep',
 'dhdr_rut_emisor',
 'dhdr_folio',
 'dtdc_codigo',
 'dhdr_dv_emisor',
 'dhdr_dv_recep',
 'dhdr_mnt_total',
 'dhdr_iva',
 'avg_dhdr_mnt_total_emisor',
 'stddev_dhdr_mnt_total_emisor',
 'avg_dhdr_iva_emisor',
 'stddev_dhdr_iva_emisor',
 'avg_dhdr_mnt_total_receptor',
 'stddev_dhdr_mnt_total_receptor',
 'avg_dhdr_iva_receptor',
 'stddev_dhdr_iva_receptor',
 'anio',
 'mes',
 'dia',
 'hora',
 'es_fin_de_semana',
 'bloque_horario',
 'dia_semana',
 'semana_mes']

## Union de documentos con contribuyentes

In [7]:
# 1. Renombrar las columnas del DataFrame contribuyentes para el emisor
contribuyentes_emisor = contribuyentes.select(
    *[F.col(c).alias(f"emis_{c}") for c in contribuyentes.columns]
)


In [8]:
# 2. Hacer el primer join para el emisor
dte_emisor = dte.join(
    contribuyentes_emisor,
    (dte["dhdr_rut_emisor"] == contribuyentes_emisor["emis_CONT_RUT"]) &
    (dte["dhdr_dv_emisor"] == contribuyentes_emisor["emis_CONT_DV"]),
    how="left"
)

In [9]:
dte_emisor.columns

['dhdr_rut_recep',
 'dhdr_rut_emisor',
 'dhdr_folio',
 'dtdc_codigo',
 'dhdr_dv_emisor',
 'dhdr_dv_recep',
 'dhdr_mnt_total',
 'dhdr_iva',
 'avg_dhdr_mnt_total_emisor',
 'stddev_dhdr_mnt_total_emisor',
 'avg_dhdr_iva_emisor',
 'stddev_dhdr_iva_emisor',
 'avg_dhdr_mnt_total_receptor',
 'stddev_dhdr_mnt_total_receptor',
 'avg_dhdr_iva_receptor',
 'stddev_dhdr_iva_receptor',
 'anio',
 'mes',
 'dia',
 'hora',
 'es_fin_de_semana',
 'bloque_horario',
 'dia_semana',
 'semana_mes',
 'emis_CONT_RUT',
 'emis_CONT_DV',
 'emis_INICIO_SEGMENTO',
 'emis_ES_EMPRESA',
 'emis_ES_PERSONA',
 'emis_Alerta_1019',
 'emis_Alerta_2250',
 'emis_Alerta_400X',
 'emis_Alerta_4110',
 'emis_Alerta_4111',
 'emis_Alerta_4112',
 'emis_Alerta_4113',
 'emis_Alerta_52',
 'emis_Alerta_5201',
 'emis_Alerta_5203',
 'emis_Alerta_53',
 'emis_Alerta_5301',
 'emis_UNOP_UNIDAD_GRAN_CONT',
 'emis_UNOP_COD_REGIONAL',
 'emis_NEGO_IND_EXPORTADOR_VO',
 'emis_NEGO_IND_PRIMERA_EXP_VO',
 'emis_NEGO_IND_VERIFICACION_VO',
 'emis_NEGO_NRO_

In [10]:
# 3. Renombrar las columnas del DataFrame contribuyentes para el receptor
contribuyentes_receptor = contribuyentes.select(
    *[F.col(c).alias(f"recep_{c}") for c in contribuyentes.columns ]
)

In [11]:
# 4. Hacer el segundo join para el receptor
dte_final = dte_emisor.join(
    contribuyentes_receptor,
    (dte_emisor["dhdr_rut_recep"] == contribuyentes_receptor["recep_CONT_RUT"]) &
    (dte_emisor["dhdr_dv_recep"] == contribuyentes_receptor["recep_CONT_DV"]),
    how="left"
)

In [12]:
dte_final.columns

['dhdr_rut_recep',
 'dhdr_rut_emisor',
 'dhdr_folio',
 'dtdc_codigo',
 'dhdr_dv_emisor',
 'dhdr_dv_recep',
 'dhdr_mnt_total',
 'dhdr_iva',
 'avg_dhdr_mnt_total_emisor',
 'stddev_dhdr_mnt_total_emisor',
 'avg_dhdr_iva_emisor',
 'stddev_dhdr_iva_emisor',
 'avg_dhdr_mnt_total_receptor',
 'stddev_dhdr_mnt_total_receptor',
 'avg_dhdr_iva_receptor',
 'stddev_dhdr_iva_receptor',
 'anio',
 'mes',
 'dia',
 'hora',
 'es_fin_de_semana',
 'bloque_horario',
 'dia_semana',
 'semana_mes',
 'emis_CONT_RUT',
 'emis_CONT_DV',
 'emis_INICIO_SEGMENTO',
 'emis_ES_EMPRESA',
 'emis_ES_PERSONA',
 'emis_Alerta_1019',
 'emis_Alerta_2250',
 'emis_Alerta_400X',
 'emis_Alerta_4110',
 'emis_Alerta_4111',
 'emis_Alerta_4112',
 'emis_Alerta_4113',
 'emis_Alerta_52',
 'emis_Alerta_5201',
 'emis_Alerta_5203',
 'emis_Alerta_53',
 'emis_Alerta_5301',
 'emis_UNOP_UNIDAD_GRAN_CONT',
 'emis_UNOP_COD_REGIONAL',
 'emis_NEGO_IND_EXPORTADOR_VO',
 'emis_NEGO_IND_PRIMERA_EXP_VO',
 'emis_NEGO_IND_VERIFICACION_VO',
 'emis_NEGO_NRO_

In [13]:
dte_final.columns

['dhdr_rut_recep',
 'dhdr_rut_emisor',
 'dhdr_folio',
 'dtdc_codigo',
 'dhdr_dv_emisor',
 'dhdr_dv_recep',
 'dhdr_mnt_total',
 'dhdr_iva',
 'avg_dhdr_mnt_total_emisor',
 'stddev_dhdr_mnt_total_emisor',
 'avg_dhdr_iva_emisor',
 'stddev_dhdr_iva_emisor',
 'avg_dhdr_mnt_total_receptor',
 'stddev_dhdr_mnt_total_receptor',
 'avg_dhdr_iva_receptor',
 'stddev_dhdr_iva_receptor',
 'anio',
 'mes',
 'dia',
 'hora',
 'es_fin_de_semana',
 'bloque_horario',
 'dia_semana',
 'semana_mes',
 'emis_CONT_RUT',
 'emis_CONT_DV',
 'emis_INICIO_SEGMENTO',
 'emis_ES_EMPRESA',
 'emis_ES_PERSONA',
 'emis_Alerta_1019',
 'emis_Alerta_2250',
 'emis_Alerta_400X',
 'emis_Alerta_4110',
 'emis_Alerta_4111',
 'emis_Alerta_4112',
 'emis_Alerta_4113',
 'emis_Alerta_52',
 'emis_Alerta_5201',
 'emis_Alerta_5203',
 'emis_Alerta_53',
 'emis_Alerta_5301',
 'emis_UNOP_UNIDAD_GRAN_CONT',
 'emis_UNOP_COD_REGIONAL',
 'emis_NEGO_IND_EXPORTADOR_VO',
 'emis_NEGO_IND_PRIMERA_EXP_VO',
 'emis_NEGO_IND_VERIFICACION_VO',
 'emis_NEGO_NRO_

In [14]:
#Se elige solo algunas columnas a guardar(por temas de tamanio del dataset original, no se guardara todas las variables. Eso puede ser modificado

numerical_columns = [
    'dhdr_mnt_total', 'dhdr_iva',
    'recep_COCO_IMP_VENTAS_IVA',
    'recep_COCO_IMP_VENTAS_TRANSPORTE',
    'recep_COCO_MTO_DEV_SOLICITADA_F22',
    'recep_COCO_MTO_VENTAS',
    'recep_NEGO_NRO_FACTURAS_6MESES_VO',
    'emis_COCO_IMP_VENTAS_IVA',
    'emis_COCO_IMP_VENTAS_TRANSPORTE',
    'emis_COCO_MTO_DEV_SOLICITADA_F22',
    'emis_COCO_MTO_VENTAS',
    'emis_NEGO_NRO_FACTURAS_6MESES_VO',
    'avg_dhdr_mnt_total_emisor',
     'stddev_dhdr_mnt_total_emisor',
     'avg_dhdr_iva_emisor',
     'stddev_dhdr_iva_emisor',
     'avg_dhdr_mnt_total_receptor',
     'stddev_dhdr_mnt_total_receptor',
     'avg_dhdr_iva_receptor',
     'stddev_dhdr_iva_receptor'
]


categorical_columns = [
    'es_fin_de_semana',
     'bloque_horario', 'dia_semana', 'semana_mes',
    'emis_INICIO_SEGMENTO', 'emis_ACEC_DES_SUBRUBRO_PPAL', 'emis_Alerta_1019', 'emis_Alerta_2250',
    'emis_Alerta_400X', 'emis_Alerta_4110', 'emis_Alerta_4111', 'emis_Alerta_4112',
    'emis_Alerta_4113', 'emis_Alerta_52', 
    'emis_Alerta_5201',
    'emis_Alerta_5203',
    'emis_Alerta_53',
    'emis_Alerta_5301',
    'recep_INICIO_SEGMENTO', 'recep_ACEC_DES_SUBRUBRO_PPAL', 'recep_Alerta_1019', 'recep_Alerta_2250',
    'recep_Alerta_400X', 'recep_Alerta_4110', 'recep_Alerta_4111', 'recep_Alerta_4112',
    'recep_Alerta_4113', 'recep_Alerta_52',
    'recep_Alerta_5201',
    'recep_Alerta_5203',
   'recep_Alerta_5301',
    'recep_Alerta_53'
]

columnas_a_conservar = numerical_columns + categorical_columns + ['dhdr_folio']

In [15]:
dte_final = dte_final.select(*columnas_a_conservar)

In [16]:
# 7. Eliminar las columnas de RUT y DV originales
dataset_final = dte_final.drop('dtdc_codigo',
 'dhdr_rut_emisor',
 'dhdr_dv_emisor',
 'dhdr_rut_recep',
 'dhdr_dv_recep',"dhdr_fch_emis","dhdr_tmst_firma")


In [17]:
dataset_final.printSchema()

root
 |-- dhdr_mnt_total: decimal(22,5) (nullable = true)
 |-- dhdr_iva: long (nullable = true)
 |-- recep_COCO_IMP_VENTAS_IVA: double (nullable = true)
 |-- recep_COCO_IMP_VENTAS_TRANSPORTE: double (nullable = true)
 |-- recep_COCO_MTO_DEV_SOLICITADA_F22: string (nullable = true)
 |-- recep_COCO_MTO_VENTAS: double (nullable = true)
 |-- recep_NEGO_NRO_FACTURAS_6MESES_VO: integer (nullable = true)
 |-- emis_COCO_IMP_VENTAS_IVA: double (nullable = true)
 |-- emis_COCO_IMP_VENTAS_TRANSPORTE: double (nullable = true)
 |-- emis_COCO_MTO_DEV_SOLICITADA_F22: string (nullable = true)
 |-- emis_COCO_MTO_VENTAS: double (nullable = true)
 |-- emis_NEGO_NRO_FACTURAS_6MESES_VO: integer (nullable = true)
 |-- avg_dhdr_mnt_total_emisor: decimal(26,9) (nullable = true)
 |-- stddev_dhdr_mnt_total_emisor: double (nullable = true)
 |-- avg_dhdr_iva_emisor: double (nullable = true)
 |-- stddev_dhdr_iva_emisor: double (nullable = true)
 |-- avg_dhdr_mnt_total_receptor: decimal(26,9) (nullable = true)
 |--

In [18]:
dataset_final.count()

3450520

In [19]:
dataset_final.write.mode('overwrite').format("parquet").save("abfs://data@datalakesii.dfs.core.windows.net/DatosOrigen/lr-629/APA/Analisis_factura/dataset_final_estratificado")


In [20]:
spark.stop()